<a href="https://colab.research.google.com/github/Eunjung-Cho/Project/blob/master/%EA%B0%9C%EC%9D%B8/graphql_json_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json

In [ ]:
# Json 파일 불러오기
with open('/content/graphql_brackets.json', 'r') as file:
    json_data = json.load(file)

In [ ]:
# 관련 필드 추출
calculated_fields = json_data['data']['calculatedFields']
df = pd.DataFrame(calculated_fields, columns=[
    'name',
    'description',
    'dataType',
    'formula',
    'referencedByCalculations',
    'upstreamDatabases',
    'downstreamSheets',
    'downstreamDashboards',
    'downstreamWorkbooks'])


In [ ]:
#안에있는 딕셔너리 한번더 뽑아내기
df['해당필드를 참조한 지표'] = df['referencedByCalculations'].apply(lambda x: ', '.join([item['name'] for item in x]) if len(x) > 0 else None)
df['데이터원본명']=df['upstreamDatabases'].apply(lambda x: ', '.join([item['name'] for item in x]) if len(x) > 0 else None)
df['사용된 시트명']=df['downstreamSheets'].apply(lambda x: ', '.join([item['name'] for item in x]) if len(x) > 0 else None)
df['사용된 대시보드명']=df['downstreamDashboards'].apply(lambda x: ', '.join([item['name'] for item in x]) if len(x) > 0 else None)
df['사용된 통합문서명']=df['downstreamWorkbooks'].apply(lambda x: ', '.join([item['name'] for item in x]) if len(x) > 0 else None)




# 원래 필드 drop하기
df.drop('referencedByCalculations', axis=1, inplace=True)
df.drop('upstreamDatabases', axis=1, inplace=True)
df.drop('downstreamSheets', axis=1, inplace=True)
df.drop('downstreamDashboards', axis=1, inplace=True)
df.drop('downstreamWorkbooks', axis=1, inplace=True)


In [ ]:
#컬럼명 변경
df = df.rename(columns={'name':'계산된필드명', 'description':'설명', 'dataType':'데이터유형', 'formula':'계산식'})

In [ ]:
#컬럼 순서 바꾸기
df = df.reindex(columns=[
    '데이터원본명',
    '사용된 통합문서명',
    '사용된 대시보드명',
    '사용된 시트명',
    '계산된필드명',
    '데이터유형',
    '계산식',
    '설명',
    '해당필드를 참조한 지표'
])

In [ ]:
df


,데이터원본명,사용된 통합문서명,사용된 대시보드명,사용된 시트명,계산된필드명,데이터유형,계산식,설명,해당필드를 참조한 지표
0,116.121.60.167:1521,상품_연차시즌별 매출 구성비 분석,D:연차 시즌별 매출 구성비 분석,전일,전년매출(전일),INTEGER,"IF DATEADD('day',-1,DATEADD('year',-1,[기준일])) ...",None,"신장율(전일), 전년판가율(전일)"
1,f26508d9-0915-48cb-848c-8084942d92cc,None,None,None,1인당 GDP,REAL,[GDP]/[모집단 총계],None,None
2,116.121.60.167:1521,None,None,None,판매일자(날짜),DATE,"DATE(DATEPARSE('YYYYMMdd',[판매일자]))",None,None
3,None,상품_아이템별 정성자료,D:아이템별 정성자료,S:아이템별 정성자료(사이즈),판매 계산,REAL,sum([기준일 판매수량])/sum([전체 판매수량]),None,None
4,ft,패션트렌드,D:키워드분석,S:연간비교,월간,DATETIME,"datetrunc('month', DATEPARSE ('yyyyMM-dd', '19...",None,None
...,...,...,...,...,...,...,...,...,...
983,116.121.60.167:1521,상품_연차시즌별 매출 구성비 분석,D:연차 시즌별 매출 구성비 분석,기준일,전년매출(기준일),INTEGER,"IF [판매일자(날짜)] = \r\nDATEADD('year',-1,[기준일]) t...",None,신장율(기준일)
984,None,Admin Insights Starter,User Drilldown,Volume by Site Role,% Occupied Licenses By Type,STRING,IF ATTR([License Type]) = 'Creator' or ATTR([L...,None,None
985,동광데모데이터.accdb,신상판매분석,D:신상 판매 분석,"S:신상 판매 분석, S:신상 판매 분석 (품번)",판가율,REAL,sum([실판매금액])/sum([판매수량]*[최초판매가]),None,None
986,None,Calendar Circle,대시보드 1,Circled,X,REAL,SIN(RADIANS([Item]))*[Distance],None,None


In [ ]:
# csv로 저장
df.to_csv('calculatedFields.csv', index=False, encoding='utf-8-sig')